In [ ]:
# 왕실의 기사 대결 - 코드트리
# tc 38에서 막힘


L, N, Q = list(map(int, input().split()))

mat = [[0 for i in range(L)] for j in range(L)]

hamjung = []
for i in range(L):
    mat[i] = list(map(int, input().split()))
    for j in range(L):
        if mat[i][j] == 1:
            hamjung.append([i, j])

knightList = []
knightList_ori_hp = []
for i in range(N):
    r, c, h, w, k = list(map(int, input().split()))
    knightList.append([r - 1, c - 1, h, w, k, False])  # 마지막은 사라졌는지 여부
    knightList_ori_hp.append(k)

# knightMat 만들기
knightMat = [[-1 for i in range(L)] for j in range(L)]
for i in range(N):
    r, c, h, w, k, _ = knightList[i]

    for each_r in range(r, r + h):
        for each_c in range(c, c + w):
            knightMat[each_r][each_c] = i

# for i in knightMat:
#     print(i)

kingList = []
for i in range(Q):
    kingList.append(list(map(int, input().split())))

dys = [-1, 0, 1, 0]
dxs = [0, 1, 0, -1]


def in_range(y, x):
    return 0 <= y and y < L and 0 <= x and x < L


def bfs(kn_idx, kn_dir):
    # 선택된 기사의 pose 처음꺼만 queue에 추가
    # 하나씩 꺼내서 bfs 진행
    # 조건 : 기사가 가고자 하는 방향으로만 갈것 , 근데 같은 기사끼리는 방향달라도 갈수있음
    # bfs 하는 중에 최대 행이나 열이 (방향 고려해서 생각) L 이거나
    # 가는 방향에 벽이 있으면 못감
    # flag 바꾸고 다음 명령 진행

    r, c, h, w, k, _ = knightList[kn_idx]

    queue = [[r, c]]
    visited = [[r, c]]
    bfsMat = [[0 for i in range(L)] for j in range(L)]
    bfsMat[r][c] = 1

    # max_row
    changedKnight = []
    while queue:
        cy, cx = queue.pop(0)
        changedKnight.append(knightMat[cy][cx])

        dir_idx = -1
        for dy, dx in zip(dys, dxs):
            dir_idx += 1

            ny, nx = cy + dy, cx + dx

            if (not in_range(ny, nx)):
                if (kn_dir == 0 and ny == -1
                        or kn_dir == 1 and nx == L
                        or kn_dir == 2 and ny == L
                        or kn_dir == 3 and nx == -1):
                    return False, _
                else:
                    continue
            elif (mat[ny][nx] == 2 and dir_idx == kn_dir):
                return False, _
            elif (knightMat[ny][nx] == -1 or [ny, nx] in visited):
                continue
            elif (knightMat[cy][cx] != knightMat[ny][nx] and dir_idx != kn_dir):
                continue

            bfsMat[ny][nx] = bfsMat[cy][cx] + 1
            queue.append([ny, nx])
            visited.append([ny, nx])

    return True, list(set(changedKnight))


# print()
for round, kingCmd in enumerate(kingList):

    ## 기사 이동
    kn_idx, kn_dir = kingCmd
    kn_idx -= 1

    # 사라진 기사인지 확인
    if (knightList[kn_idx][-1] == True):
        continue

    res, changedKnight = bfs(kn_idx, kn_dir)
    if (res == False):
        # print('res False')
        continue

    # print(changedKnight)

    # 갈 수 있다면
    # bfs 1이상인 애들을 그대로 1씩 옮기기 (knightMat 업데이트)
    # knightList 업데이트

    go_y, go_x = 0, 0
    if (kn_dir == 0):
        go_y, go_x = -1, 0
    elif (kn_dir == 1):
        go_y, go_x = 0, 1
    elif (kn_dir == 2):
        go_y, go_x = 1, 0
    elif (kn_dir == 3):
        go_y, go_x = 0, -1

    # print(kn_dir, go_y, go_x)
    knightMat = [[-1 for i in range(L)] for j in range(L)]
    for i in range(len(knightList)):
        if (i in changedKnight):
            knightList[i][0] += go_y
            knightList[i][1] += go_x

        r, c, h, w, k, _ = knightList[i]
        # print(r, c, h, w, k, _)

        for each_r in range(r, r + h):
            for each_c in range(c, c + w):
                knightMat[each_r][each_c] = i

    # for i in knightMat:
    #     print(i)
    # print()

    # 함정 피해
    # mat 1 자리에 knightMat 에 누구 있나 확인해서 체력 깎기
    # 단,  명령을 받은 기사는 피해를 입지 않도록
    for eachHamjung in hamjung:
        r, c = eachHamjung
        knightIdx = knightMat[r][c]
        if (knightIdx >= 0 and knightIdx != kn_idx and knightIdx in changedKnight):
            knightList[knightIdx][-2] -= 1
            # print(knightIdx)

    # 각 기사의 남은 체력이 0 이하인 경우 knightMat에서 지우기 (해당 자리 -1로 만들기)
    # knightList[kn_idx][-1] = True로 바꾸기
    for i in range(N):
        each = knightList[i]
        if (each[-2] <= 0 and each[-1] == False):
            knightList[i][-1] = True

            r, c, h, w, k, _ = knightList[i]
            for each_r in range(r, r + h):
                for each_c in range(c, c + w):
                    knightMat[each_r][each_c] = -1

    # for i in knightList:
    #     print(i[-2])
    # print(kn_idx, kn_dir)
    # for i in knightMat:
    #     print(i)
    # print()

    # break

damages = 0
for i in range(N):
    if (knightList[i][-1] == True):
        continue

    diff = knightList_ori_hp[i] - knightList[i][-2]
    damages += diff

print(damages)